In [18]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.optimizers.schedules import ExponentialDecay
# from tensorflow.keras.losses import BinaryCrossentropyLoss
from tqdm import trange


class FullyConnectedNet(Model):
    def __init__(self, hidden_dim, output_dim, activation='relu'):
        super(FullyConnectedNet, self).__init__()
        self.dense1 = Dense(hidden_dim, activation=activation)
        self.out = Dense(output_dim, activation='sigmoid')
    def call(self,x):
        x = self.dense1(x)
        x = self.out(x)
        return x


def train(model, dataset, loss_obj, num_epochs, optimizer, train_loss, train_accuracy):
    for epoch in trange(num_epochs):
        train_loss.reset_states()
        train_accuracy.reset_states()

        for X_data, labels in dataset:
            with tf.GradientTape() as tape:
                pred = model(X_data)
                loss = loss_obj(labels, pred)
            
            grad = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grad, model.trainable_variables))

            train_loss(labels, pred)
            train_accuracy(labels, tf.squeeze(pred, axis=1))
        
        if epoch%1 == 0:
            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {train_loss.result()}, '
                f'Accuracy: {train_accuracy.result() * 100}'
            )


lr = ExponentialDecay(2, decay_steps=10, decay_rate=0.7, name="exponential_decay_schedule")
num_epochs = 50
dtype = tf.float64
model = FullyConnectedNet(2,1,'sigmoid')

loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
train_loss = tf.keras.metrics.BinaryCrossentropy()
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='binary_accuracy')

training_data = np.loadtxt("training_data.txt")
test_data = np.loadtxt("test_data.txt")
training_data = tf.convert_to_tensor(training_data, dtype=dtype)
test_data = tf.convert_to_tensor(test_data, dtype=dtype)
X_train, y_train = training_data[:,1:], training_data[:,0]
X_test, y_test = training_data[:,1:], training_data[:,0]

train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)
).shuffle(1000).batch(1)


train(model, train_ds, loss_object, num_epochs, optimizer, train_loss, train_accuracy)







  2%|▏         | 1/50 [00:35<29:21, 35.95s/it]

Epoch 1, Loss: 0.9281683564186096, Accuracy: 50.0


  4%|▍         | 2/50 [01:11<28:33, 35.70s/it]

Epoch 2, Loss: 0.9249695539474487, Accuracy: 50.0


  6%|▌         | 3/50 [01:46<27:53, 35.61s/it]

Epoch 3, Loss: 0.9249691963195801, Accuracy: 50.0


  8%|▊         | 4/50 [02:21<27:04, 35.31s/it]

Epoch 4, Loss: 0.9249691963195801, Accuracy: 50.0


  8%|▊         | 4/50 [02:25<27:54, 36.39s/it]


KeyboardInterrupt: 